In [65]:
!pip install nltk

In [1]:
import pandas as pd #работа с датафреймами- таблицами
import pymorphy2 as pm #части речи, формы
import re #регулярные выражения
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count, freeze_support
import time
from IPython.display import display 
import nltk

In [2]:
#логер
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [3]:
places_df = pd.read_csv('../data/places_GP.csv')
df = pd.read_csv('../data/comments_filled.csv')
words = pd.read_csv('../data/words.csv')
bigramms = pd.read_csv('../data/bigramms.csv')

In [8]:
df = pd.read_csv('../data/comments.csv')
df = df.loc[~df.text.isna()]
display(df.sample())
df.shape

C:\Users\yupes\anaconda3\envs\python-gis\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,url,id_review,reviewer_page,reviewer_level,reviewer_counts,rating,likes,date,text,photos
16222,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1133506835...,1 отзыв,0,5.0,0,3 года назад,Хороший магазин. Есть дисконтные программы.,[]


(198846, 10)

In [13]:
df.loc[df.text.str.contains('Google'), 'text'] = df.loc[df.text.str.contains('Google'), 'text'].astype(str).str.split('\n\n\(Оригинал\)\n', expand = False).str[0].str.split('\(Переведено Google\)\s').str[-1]

In [29]:
words = df['text'].str.extractall(r"(\S+)").reset_index()

In [30]:
# переименовываем колонки
words.rename(columns = {'level_0':'index_review', 0:'word'}, inplace = True)

In [31]:
# все буквы в словах делаем строчными
words.word = words.word.str.lower()

In [32]:
# создание новых столбцов, в нашем случае - индекса отзывов, 
# чтобы по этому индексу мы могли идентифицировать пренадлежность слов ко всем полям df
df['index_review']= df.index

In [33]:
# удаляем лишние символы, оставляя только слова
chars_to_remove = ['\r', '\n', '.', ')', '!', '(', ',', '-', ':', '?', '/', '"', ';']
regular_expression = '[' + re.escape (''. join (chars_to_remove)) + ']'
words.word = words.word.str.replace(regular_expression, ' ')

C:\Users\yupes\AppData\Local\Temp/ipykernel_10380/1603569367.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  words.word = words.word.str.replace(regular_expression, ' ')


In [44]:
words = pd.DataFrame(words.word.str.strip().str.split().explode()).merge(words[['index_review', 'match']], how = 'left',left_index=True, right_index=True)

In [48]:
words = words.loc[~words.word.isna()]

In [ ]:
list_morph = []
datalist = words.word.drop_duplicates()

In [50]:
# преобразование слов в начальную форму с наименованием частей речи
def add_morph(words_list):
    morph = pm.MorphAnalyzer()

    for word in log_progress(words_list, every = 1):
        try:
            m = morph.parse(word.strip())[0]
            list_morph.append({'normal_sentens':word, 'new':m.normal_form, 'tag':m.tag, 'grammema':m.tag.POS})
        except:
            continue

In [67]:
morph = pm.MorphAnalyzer()
# только нормальные граммемы
g = ['NOUN','ADJF', 'ADJS', 'COMP','VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'ADVB']

In [96]:
bigs  = []
datalist = list(zip(df.normal_sentense, df.index_review))

In [94]:
def get_bigramms(sentences):
    for sentence, index in log_progress(sentences, 1):
        s = re.sub("[^\w]", " ", sentence)
        bigrm = nltk.bigrams(s.lower().split(" "))
        for l in list(bigrm):
            if l[0] != '' and l[1]!='':
                m = morph.parse(l[0])[0].tag.POS
                m2 = morph.parse(l[1])[0].tag.POS
                if str(m) in g and str(m2) in g:
                    bigs.extend([[index, l, l[0], l[1], m, m2]])

In [11]:
d_sent = []
datalist = list(zip(df.normal_sentense, df.index_review))

In [12]:
if __name__== "__main__":
    start = time.time()
    freeze_support()
    pool = Pool()
    
    ITERATION_COUNT = cpu_count()-1

    count_per_iteration = len(datalist) / float(ITERATION_COUNT)

    for i in range(0, ITERATION_COUNT):
        list_start = int(count_per_iteration * i)
        list_end = int(count_per_iteration * (i+1))
        pool.apply_async(get_sentiment, [datalist[list_start:list_end]])

66193
269107
269761
306041


In [108]:
b_df = pd.DataFrame(bigs).rename(columns = {0:'index_review', 1:'bigramm', 2:'word1', 3:'word2', 4:'tag1', 5:'tag2'})

In [48]:
b_df = bigramms.merge(df_temp[['index_review', 'url', 'score', 'positive', 'negative', 'comparative']], how = 'left', on = 'index_review').merge(places_df[['category', 'url']], how = 'left', on = 'url')

In [49]:
b_df

,index_review,bigramm,word1,word2,tag1,tag2,url,score,positive,negative,comparative,category
0,207236,"('отличный', 'магазин')",отличный,магазин,ADJF,NOUN,https://www.google.com/maps/place/?q=place_id:...,8.3,8.3,0.0,2.075000,Магазин
1,207237,"('ужасный', 'обслуживание')",ужасный,обслуживание,ADJF,NOUN,https://www.google.com/maps/place/?q=place_id:...,-5.0,5.0,-10.0,-0.098039,Магазин
2,207237,"('обслуживание', 'магазин')",обслуживание,магазин,NOUN,NOUN,https://www.google.com/maps/place/?q=place_id:...,-5.0,5.0,-10.0,-0.098039,Магазин
3,207237,"('магазин', 'продавец')",магазин,продавец,NOUN,NOUN,https://www.google.com/maps/place/?q=place_id:...,-5.0,5.0,-10.0,-0.098039,Магазин
4,207237,"('продавец', 'вообще')",продавец,вообще,NOUN,ADVB,https://www.google.com/maps/place/?q=place_id:...,-5.0,5.0,-10.0,-0.098039,Магазин
...,...,...,...,...,...,...,...,...,...,...,...,...
1279680,336213,"('есть', 'выбор')",есть,выбор,INFN,NOUN,https://www.google.com/maps/place/?q=place_id:...,5.0,5.0,0.0,1.250000,Магазин
1279681,336214,"('большой', 'выбор')",большой,выбор,ADJF,NOUN,https://www.google.com/maps/place/?q=place_id:...,5.0,5.0,0.0,0.714286,Магазин
1279682,336214,"('выбор', 'качественный')",выбор,качественный,NOUN,ADJF,https://www.google.com/maps/place/?q=place_id:...,5.0,5.0,0.0,0.714286,Магазин
1279683,336214,"('качественный', 'обувь')",качественный,обувь,ADJF,NOUN,https://www.google.com/maps/place/?q=place_id:...,5.0,5.0,0.0,0.714286,Магазин


In [50]:
b_df.to_csv('../data/bigramms.csv', index = False)

In [52]:
len(datalist), len(list_morph)

(138812, 138812)

In [58]:
words = words.merge(pd.DataFrame(list_morph).rename(columns = {'normal_sentens':'word', 'new':'normal_form'}), how = 'left', on = 'word')

In [30]:
words = words.merge(df[['index_review', 'url']], how = 'left', on = 'index_review')

In [34]:
words = words.merge(places_df[['category', 'url']], how = 'left', on = 'url')

In [42]:
words = words.merge(df_temp[['index_review', 'score', 'positive', 'negative', 'comparative']], how = 'left', on = 'index_review')

In [43]:
words.to_csv('../data/words.csv', index = False)

In [73]:
df['normal_sentense'] = df.index_review.apply(lambda x: ' '.join(words.loc[words.index_review==x, 'normal_form']))

# Определение сентиментов

In [9]:
df['text'] = df['text'].str.replace('ё', 'е')
df['normal_sentense'] = df['normal_sentense'].str.replace('ё', 'е')

In [10]:
df.loc[df.text.str.contains('ё')]

,url,id_review,reviewer_page,reviewer_level,reviewer_counts,rating,likes,date,text,photos,index_review,normal_sentense


In [5]:
# код для семантического анализа

import re
import os
import csv
from collections import defaultdict


class Sentimental(object):
    def __init__(self, word_list=None, negation=None):
        self.word_list = {}
        self.negations = set()

        for wl_filename in self.__to_arg_list(word_list):
            self.load_word_list(wl_filename)
        for negations_filename in self.__to_arg_list(negation):
            self.load_neagations(negations_filename)

        self.__negation_skip = {'a', 'an', 'so', 'too', 'я', 'мне'}

    @staticmethod
    def __to_arg_list(obj):
        if obj is not None:
            if not isinstance(obj, list):
                obj = [obj]
        else:
            obj = []
        return obj

    def __is_prefixed_by_negation(self, token_idx, tokens):
        #   True if i != 0 and tokens[i - 1] in self.negations else False
        prev_idx = token_idx - 1
        if tokens[prev_idx] in self.__negation_skip:
            prev_idx -= 1

        is_prefixed = False
        if token_idx > 0 and prev_idx >= 0 and tokens[prev_idx] in self.negations:
            is_prefixed = True

        return is_prefixed

    def load_neagations(self, filename):
        with open(filename,'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            negations = set([row['token'] for row in reader])
        self.negations |= negations

    def load_word_list(self, filename):
        with open(filename, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            word_list = {row['word']: float(row['score']) for row in reader}
        self.word_list.update(word_list)

    def analyze(self, sentence):
        sentence_clean = re.sub(r'[^\w ]', ' ', sentence.lower())
        tokens = sentence_clean.split()

        scores = defaultdict(float)
        words = defaultdict(list)
        comparative = 0

        for i, token in enumerate(tokens):
            is_prefixed_by_negation = self.__is_prefixed_by_negation(i, tokens)
            if token in self.word_list and not is_prefixed_by_negation:
                score = self.word_list[token]

                score_type = 'negative' if score < 0 else 'positive'
                scores[score_type] += score
                words[score_type].append(token)

        if len(tokens) > 0:
            comparative = (scores['positive'] + scores['negative']) / len(tokens)

        result = {
            'score': scores['positive'] + scores['negative'],
            'positive': scores['positive'],
            'negative': scores['negative'],
            'comparative': comparative,
        }

        return result
sent = Sentimental(word_list=['D:/projects/word_list/afinn.csv', 'D:/projects/word_list/russian.csv'], negation='D:/projects/word_list/negations.csv')

In [6]:
def get_sentiment(sentences):
    for sentence, index in log_progress(sentences, 1):
        try:
            analyze = sent.analyze(sentence)
            list_sent = {}
            for attr in ['score', 'positive', 'negative', 'comparative']:
                list_sent.update({attr: analyze.get(attr)})
            list_sent.update({'index_review':index})
            d_sent.append(list_sent)
        except:
            print(index)

In [27]:
df_temp = df.merge(pd.DataFrame(d_sent), how = 'left', on = 'index_review').copy()

In [47]:
df_temp[['rating', 'positive', 'negative', 'score', 'text']].sample(6)

,rating,positive,negative,score,text
141456,5.0,4.2,0.0,4.2,Сергей оказался специалистом самого высоко уро...
147905,NaN,10.0,-5.0,5.0,В принципе хороший отель.\nУдивило грязное нап...
59875,5.0,3.3,0.0,3.3,Отличный парк
132739,1.0,8.3,-10.0,-1.7,Никому не рекомендую данный магазин и фирму iq...
114806,5.0,8.3,0.0,8.3,Мощный магазин! Представленны все виды товаров...
23825,3.0,0.0,-7.5,-7.5,Заставляют надевать маски для показухи


In [26]:
df.shape, pd.DataFrame(d_sent).shape

((198846, 12), (198819, 5))

In [28]:
df_temp.loc[df_temp.score.isna()]

,url,id_review,reviewer_page,reviewer_level,reviewer_counts,rating,likes,date,text,photos,index_review,normal_sentense,score,positive,negative,comparative
35488,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1150033066...,1 отзыв,0,5.0,0,3 года назад,:) :) :),[],36027,NaN,NaN,NaN,NaN,NaN
55773,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1125891516...,105 отзывов,0,5.0,0,4 года назад,.,[],63675,NaN,NaN,NaN,NaN,NaN
57024,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1061748156...,Местный эксперт,57,4.0,0,2 года назад,!!!!!,[],66193,NaN,NaN,NaN,NaN,NaN
71562,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSURndHR5R1ZnEAE,https://www.google.com/maps/contrib/1125891516...,105 отзывов,0,4.0,0,4 года назад,.,[],98950,NaN,NaN,NaN,NaN,NaN
92976,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSURRemVEQUVnEAE,https://www.google.com/maps/contrib/1046592736...,Местный эксперт,89,NaN,0,NaN,............,[],156019,NaN,NaN,NaN,NaN,NaN
96622,https://www.google.com/maps/place/?q=place_id:...,ChdDSUhNMG9nS0VJQ0FnSUNndlo2dnNnRRAB,https://www.google.com/maps/contrib/1086182772...,Местный эксперт,41,3.0,0,3 года назад,)) ((,[],165133,NaN,NaN,NaN,NaN,NaN
101593,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSUNBaGNHcktnEAE,https://www.google.com/maps/contrib/1167517142...,4 отзыва,0,5.0,2,3 года назад,/),[],177843,NaN,NaN,NaN,NaN,NaN
101973,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSUNBMHFXWEFREAE,https://www.google.com/maps/contrib/1134721627...,Местный эксперт,48,NaN,0,NaN,.,[],178854,NaN,NaN,NaN,NaN,NaN
103947,https://www.google.com/maps/place/?q=place_id:...,ChdDSUhNMG9nS0VJQ0FnSURndHFyaTdnRRAB,https://www.google.com/maps/contrib/1125891516...,105 отзывов,0,5.0,0,4 года назад,.,[],183736,NaN,NaN,NaN,NaN,NaN
105571,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSUNncm9pYUpREAE,https://www.google.com/maps/contrib/1049664090...,1 отзыв,0,NaN,1,NaN,..,[],188003,NaN,NaN,NaN,NaN,NaN


In [199]:
df_temp.loc[(df_temp.rating==1) &(df_temp.score>20)]

,url,id_review,reviewer_page,reviewer_level,reviewer_counts,rating,likes,date,text,photos,index_review,normal_sentense,score,positive,negative,comparative
5699,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1027713177...,3 отзыва,0,1.0,0,3 месяца назад,"Отличный парк, у нас в оренбурге таких нет. Пр...",['https://lh5.googleusercontent.com/p/AF1QipNF...,5699,отличный парк у мы в оренбург такой нет прекра...,25.8,25.8,0.0,1.290000
10386,https://www.google.com/maps/place/?q=place_id:...,NaN,https://www.google.com/maps/contrib/1012785602...,26 отзывов,0,1.0,0,3 месяца назад,С главной аллее убрали все ларьки((( очень люб...,[],10386,с главный аллея убрать всё ларёк очень любить ...,24.2,24.2,0.0,0.504167
75929,https://www.google.com/maps/place/?q=place_id:...,ChdDSUhNMG9nS0VJQ0FnSUNnMl9hWWlBRRAB,https://www.google.com/maps/contrib/1007046593...,5 отзывов,0,1.0,6,4 года назад,"(Бываем там в рабочие дни после 22.00, людей в...",[],110970,бывать там в рабочий день после 22 00 человек ...,30.0,30.0,0.0,0.483871
76883,https://www.google.com/maps/place/?q=place_id:...,ChdDSUhNMG9nS0VJQ0FnSURTNmFtMm1RRRAB,https://www.google.com/maps/contrib/1100827422...,13 отзывов,0,1.0,0,9 месяцев назад,"Во время прогулок по ТЦ любили , что-нибудь по...",[],113776,в время прогулка по тц любить что нибыть покуш...,24.3,42.5,-18.2,0.185496
77135,https://www.google.com/maps/place/?q=place_id:...,ChdDSUhNMG9nS0VJQ0FnSURLMW9DZTJRRRAB,https://www.google.com/maps/contrib/1107441934...,Местный эксперт,10,1.0,0,5 месяцев назад,Сразу сделаю оговорку - мой отзыв будет о рабо...,[],114477,сразу сделать оговорка мой отзыв быть о работа...,35.1,59.3,-24.2,0.063243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193794,https://www.google.com/maps/place/?q=place_id:...,ChdDSUhNMG9nS0VJQ0FnSURhcDlEOXJRRRAB,https://www.google.com/maps/contrib/1049459938...,1 отзыв,0,1.0,0,месяц назад,"Очень хорошая компания, работаю и буду работат...",[],385464,очень хороший компания работать и быть работат...,23.3,23.3,0.0,1.059091
194440,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSURJM2VfWGFnEAE,https://www.google.com/maps/contrib/1013734377...,Местный эксперт,65,1.0,3,2 года назад,В мае 2018 купили жене очки в титановой оправе...,[],386724,в май 2018 купить жена очки в титановый оправа...,26.0,36.0,-10.0,0.088737
196189,https://www.google.com/maps/place/?q=place_id:...,ChdDSUhNMG9nS0VJQ0FnSURnaGFpMHlBRRAB,https://www.google.com/maps/contrib/1126485078...,1 отзыв,0,1.0,8,4 года назад,"На первый взгляд сад очень хороший, доброжелат...",[],390150,на первый взгляд сад очень хороший доброжелате...,26.6,45.8,-19.2,0.211111
197342,https://www.google.com/maps/place/?q=place_id:...,ChZDSUhNMG9nS0VJQ0FnSUN5OThPZ05nEAE,https://www.google.com/maps/contrib/1057476348...,2 отзыва,0,1.0,1,7 месяцев назад,Заказала через Яндекс услуги. Работы по исправ...,[],392507,заказать через яндекс услуга работа по исправл...,25.0,30.0,-5.0,0.284091


In [144]:
sent.analyze('отличный торговый центр гигантский ассортимент многий вид еда который нет в типичный российский магазин мочь сказать о это только положительный …')

{'score': 8.3,
 'positive': 8.3,
 'negative': 0.0,
 'comparative': 0.41500000000000004}

In [166]:
get_sentiment(datalist[:2])

есть доставка большой ассортимент продукт вещь для авто для дом парковка отличный нравиться
score 8.3
positive 8.3
negative 0.0
comparative 0.6384615384615385
в гипер лента есть почти всë и преимущество что круглосуточно 👍
score 0.0
positive 0.0
negative 0.0
comparative 0.0


In [51]:
df_temp.to_csv('../data/comments_filled_sent.csv', index = False)